In [1]:
%pylab inline
import os
import seaborn as sns
import pickle as pkl
pal = sns.color_palette('Blues')
sns.set_context("paper", font_scale=1.5)
sns.set_style("ticks")
#sns.set(color_codes=True)
from scipy.stats import gaussian_kde
import pandas as pd
from eod import stations_cc
from scipy.stats.stats import pearsonr

Populating the interactive namespace from numpy and matplotlib


In [2]:
dic40 = stations_cc.readStation(40)
dic60 = stations_cc.readStation(60)
dic70 = stations_cc.readStation(70)

In [3]:
alog = dic40['area']
log=np.log10(alog)
drain = dic40['rain']
drain25 = drain[dic40['area']>25000]
Tmin25 = dic40['minT'][dic40['area']>25000]

thresh = np.squeeze(dic40['thresh'])
rthresh = np.array([thresh]*500).transpose()
nthresh = rthresh[dic40['area']>25000]

ok = np.where(~np.isnan(drain))
drain=drain[ok]
alog = alog[ok]
log = log[ok]
dthresh = rthresh.copy()[ok]

nok = np.where(np.isfinite(log))
drain=drain[nok]
alog = alog[nok]
#log[nok]=0.
log = log[nok]
dthresh = dthresh[nok]

In [4]:
myDicts = pkl.load( open ('/users/global/cornkle/data/OBS/MSG_TRMM_temp_pcp_300px2004-2013_new.p', 'rb'))

In [5]:
dic=myDicts[0]
p=np.array(dic['pp']) # 98th perc per MCS
t=np.array(dic['t'])  #mean T
print('Number MCSs:', p.size)
print('98th perc:', np.nanmean(np.array(dic['perc'])))
pc = p[p<81]
tm = t[p<81]

Number MCSs: 1640
98th perc: 29.5575427559


In [6]:
r=pearsonr(pc,tm)
print(r[0]**2)

0.160516181624


In [7]:
i=[-70,-90]
pos = np.where((Tmin25 <= i[0]) & (Tmin25 >= i[1]))
nex= np.sum(drain25>nthresh)
ptresh = drain25[pos]
ttresh = nthresh[pos]
prob=np.sum(ptresh>ttresh)/nex#/ptresh.size
print(prob*100)

87.4725274725


In [8]:
pos = np.where(drain>dthresh)
aarea = alog[pos]
prob = aarea[aarea>=25000].size/aarea.size
print(prob*100.)

95.78947368421052


In [27]:
path = '/users/global/cornkle/C_paper/figs/'
fig = plt.figure(figsize=(15, 15), dpi=300)
# definitions for the axes
left, width = 0.1, 0.7
bottom, height = 0.1, 0.7
bottom_h = left_h = left + width + 0.02

rect_scatter1 = [left, bottom, width, height]
rect_scatter2 = [left+width+left, bottom, width, height]
rect_scatter3 = [3*left+2*width, bottom, width, height]

rect_histx1 = [left, height+0.01, width, 0.2]
rect_histx2 = [left+width+left, height+0.01, width, 0.2]
rect_histx3 = [3*left+2*width, height+0.01, width, 0.2]


ax1 = plt.axes(rect_scatter1)
ax11 = plt.axes(rect_histx1)
xy = np.vstack([log, drain])
z = gaussian_kde(xy)(xy)
test = z / (z.max() - z.min())
mappable = ax1.scatter(log, drain, c=test, edgecolor='', cmap='viridis_r')
c = 0.8
ax1.set_ylabel('Daily rain ($mm$)')
ax1.set_xlabel('log(Cold cloud area)($km^{2}$) -40$^{\degree}C$')
ax1.vlines(np.log10(25000), np.min(drain), np.max(drain), linestyles='dashed', label='25,000 $km^{2}$', linewidth=c)
# ax1.hlines(np.mean(thresh), np.min(log), np.max(log), linestyles='dashed', label='threshold', linewidth=c)
# #ax1.set_title('$98^{th}$ perc Pcp  vs meanT per MCS')
# #cbar = fig.colorbar(mappable)
# #cbar.set_label('Normalised probability density')
# text(0.03, 0.9, 'a', transform=ax1.transAxes, fontsize=20)
# #text(0.5, 0.35, 'Probability of rain > 38 $mm$:', transform=ax1.transAxes, fontsize=10)
# #text(0.5, 0.3, 'Area > 25,000 $km^{2}$: 17.5%', transform=ax1.transAxes, fontsize=10)
# text(120, 4.5, '25,000 $km^{2}$',  fontsize=10)
# text(41, 2, '38 $mm$',  fontsize=10, rotation=90)



######################
ax2 = plt.axes(rect_scatter2)
#ax22 = plt.axes(rect_histx2)

xy = np.vstack([Tmin25, drain25])
z = gaussian_kde(xy)(xy)
test = z / (z.max() - z.min())
mappable = ax2.scatter(Tmin25, drain25,  c=test, edgecolor='', cmap='viridis_r')
ax2.set_ylabel('Daily rain ($mm$)')
ax2.set_xlabel('Local daily min. temperature ($^{\degree}C$)')
ax2.hlines(np.mean(thresh), np.min(Tmin25), np.max(Tmin25), linestyles='dashed', label='threshold', linewidth=c)
# ax1.hlines(np.mean(-60), np.min(drain25), np.max(drain25), linestyles='dashed', label='threshold', linewidth=c)
# ax1.hlines(np.mean(-70), np.min(drain25), np.max(drain25), linestyles='dashed', label='threshold', linewidth=c)
# ax1.hlines(np.mean(-80), np.min(drain25), np.max(drain25), linestyles='dashed', label='threshold', linewidth=c)
#ax1.set_title('$98^{th}$ perc Pcp  vs meanT per MCS')
#cbar = fig.colorbar(mappable)
#cbar.set_label('Normalised probability density')
text(0.03, 0.9, 'b', transform=ax1.transAxes, fontsize=20)
text(0.5, 0.9, 'Probability of rain > 38 $mm$:', transform=ax2.transAxes, fontsize=10)
text(0.6, 0.85, 'T > -60$^{\degree}C$: 9.6%', transform=ax2.transAxes, fontsize=10)
text(0.5, 0.8, '-60 > T > -70$^{\degree}C$: 9.0%', transform=ax2.transAxes, fontsize=10)
text(0.5, 0.75, '-70 > T > -80$^{\degree}C$: 15.1%', transform=ax2.transAxes, fontsize=10)
text(0.6, 0.7, 'T < -80$^{\degree}C$: 26.3%', transform=ax2.transAxes, fontsize=10)
text(41, -40, '38 $mm$',  fontsize=10, rotation=90)
#     #####################################

ax3 = plt.axes(rect_scatter3)
ax33 = plt.axes(rect_histx3)
xy = np.vstack([tm,pc])
z = gaussian_kde(xy)(xy)
test = z / (z.max() - z.min())
mappable = ax3.scatter(tm, pc, c=test, edgecolor='', cmap='viridis_r')
ax3.set_ylabel('Rain ($mm\ h^{-1}$)')
ax3.set_xlabel('Temperature ($^{\degree}C$)')
#ax1.set_title('$98^{th}$ perc Pcp  vs meanT per MCS')
text(0.03, 0.9, 'c', transform=ax3.transAxes, fontsize=20)

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", "5%", pad="3%")
plt.colorbar(mappable, cax=cax, label='Normalised kernel density')


#plt.tight_layout()
#plt.savefig(path+'station_TRMM_scatter.png')
#plt.savefig(path+'station_TRMM_scatter.eps')